# Parallel Function Calling

In [22]:
# 次の３つの値をご自身のものに置き換えてください。Replace the following three values with your resources. 
deployment = '<your-aoai-deployment-name>'
apikey = '<your-aoai-api-key>'
base = 'https://<your-aoai-resource-name>.openai.azure.com'
api_version = '2023-12-01-preview'

In [38]:
!python -m pip install openai
from openai import AzureOpenAI

In [39]:
client = AzureOpenAI(
  azure_endpoint = base, 
  api_key=apikey,  
  api_version=api_version
)

In [67]:
TOOLS = [
    {
        "type":"function",
        "function": {
            "name": "switch_airconditioner",
            "description": "エアコンの電源をスイッチします。",
            "parameters": {
                "type": "object",
                "properties": {
                    "switch": {
                        "type": "boolean",
                        "description": "オンの場合はtrue,オフの場合はfalse"
                    }
                },
                "required": ["switch"]
            }
          }
    },
    {
        "type":"function",
        "function": {
            "name": "switch_lamp",
            "description": "照明のスイッチをオン・オフします。",
            "parameters": {
                "type": "object",
                "properties": {
                    "switch": {
                        "type": "boolean",
                        "description": "オンの場合はtrue,オフの場合はfalse"
                    }
                },
                "required": ["switch"]
            }
          }
    }
]

# 並列機能呼び出しを期待する入力

In [70]:
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "エアコンと電気をつけてください。"}
    ],
    tools=TOOLS,
    tool_choice="auto",
    temperature=0,
)

if response.choices[0].finish_reason=='tool_calls':
    for tool_call in response.choices[0].message.tool_calls:
        print(tool_call.function)

Function(arguments='{"switch": true}', name='switch_airconditioner')
Function(arguments='{"switch": true}', name='switch_lamp')


# 単一機能呼び出しを期待する入力

In [71]:
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "エアコンを消してください。"}
    ],
    tools=TOOLS,
    tool_choice="auto",
    temperature=0,
)

if response.choices[0].finish_reason=='tool_calls':
    for tool_call in response.choices[0].message.tool_calls:
        print(tool_call.function)

Function(arguments='{"switch":false}', name='switch_airconditioner')
